<a href="https://colab.research.google.com/github/saotomryo/Use_MMDetection/blob/main/Use_MMDetection_for_yolox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import json
from skimage import measure
import numpy as np
print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device = ", device)

## 環境準備 MMCV MMDetectionのインストール

In [ ]:
# MMCVのインストール（バージョンに合わせて修正してください。）

!pip install https://download.openmmlab.com/mmcv/dist/cu113/torch1.10.0/mmcv_full-1.4.8-cp37-cp37m-manylinux1_x86_64.whl 

In [ ]:
HOME_PATH = "/content"

In [ ]:

!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -r requirements/build.txt
!pip install "git+https://github.com/open-mmlab/cocoapi.git#subdirectory=pycocotools"
!pip install -v -e .  # or "python setup.py develop"

## コンフィグファイルの編集

In [ ]:
%cd /content/

MMdetectionの分類結果は、COCODatasetで定義された80分類に従って、表示されます。  
表示名を変更するには、以下のファイルの'cocodataset'の部分を編集してください。  
　/content/mmdetection/mmdet/datasets/coco.py  
　/content/mmdetection/mmdet/core/evaluation/class_names.py

In [ ]:
# クラスのリストを持っている場合は、以下のように分類するクラスのリストを編集することができます。

f = open('/content/mmdetection/mmdet/datasets/coco.py', 'r', encoding='UTF-8')

data = f.read()
f.close()

from_index = data.find('CLASSES')
end_index = data.find('toothbrush') + 13

CLASSES_old = data[from_index:end_index]


CLASSES_list = 'クラス分類するリストをここに入力してください。'
num_classes = len(CLASSES_list)


CLASSES_new = "CLASSES = " + str(tuple(CLASSES_list))

data = data.replace(CLASSES_old,CLASSES_new)

f = open('/content/mmdetection/mmdet/datasets/coco.py', 'w', encoding='UTF-8')

f.write(data)

In [ ]:
f = open('/content/mmdetection/mmdet/core/evaluation/class_names.py', 'r', encoding='UTF-8')

data = f.read()
f.close()

from_index = data.find("'person', 'bicycle',")
end_index = data.find("'toothbrush'") + 12

CLASSES_old = data[from_index:end_index]
CLASSES_new = str(CLASSES_list)[1:-1]

data = data.replace(CLASSES_old,CLASSES_new)

f = open('/content/mmdetection/mmdet/core/evaluation/class_names.py', 'w', encoding='UTF-8')

f.write(data)

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/content/mmdetection/configs/yolox/yolox_tiny_8x8_300e_coco.py')

# コンフィグを変更する
# yolox

#data_root = '画像データのルートフォルダ'
data_root = '/content/'

cfg.model.bbox_head.num_classes = num_classes #分類するクラス数
#cfg.num_class = 80
cfg.data_root = data_root

# trainデータの定義
cfg.data.train.dataset.ann_file = '学習用のアノテーションファイル'
cfg.data.train.dataset.img_prefix = data_root + '学習用画像データを保存したフォルダ'

# valデータの定義
cfg.data.val.ann_file = '検証用のアノテーションファイル'
cfg.data.val.img_prefix = data_root + '検証用画像データを保存したフォルダ'

# バッチサイズ
# cfg.data.samples_per_gpu = 2 #メモリエラーが出る場合は、バッチサイズを減らしてみてください。
# cfg.data.workers_per_gpu = 2


# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


# 編集したコンフィグをファイル出力
cfg.dump('/content/yolox_tiny_8x8_300e_coco.py'')


In [ ]:
!mkdir -p "$HOME_PATH"/work_train
!python /content/mmdetection/tools/train.py \
    '/content/yolox_tiny_8x8_300e_coco.py'' \
    --work-dir "$HOME_PATH"/work_train


## 学習結果の確認

In [ ]:
from argparse import ArgumentParser
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
model = init_detector("/content/yolox_tiny_8x8_300e_coco.py'", "学習済みモデル", device=device)
result = inference_detector(model, "予測結果を確認する画像のパス")
show_result_pyplot(model, "予測結果を確認する画像のパス", result, score_thr=0.4)

## 予測タスクの実施

In [ ]:
from argparse import ArgumentParser
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
import os

testimage_path　 = "テスト画像をおいたフォルダ"

imgaes = glob(testimage_path)

# Specify the path to model config and checkpoint file
config_file = '/content/yolox_tiny_8x8_300e_coco.py''
checkpoint_file = '学習済みモデルのパスを記載する'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file,device=device)



bbox_dict1 = {}

for image in images:
    
    if os.path.exists(image):

        result = inference_detector(model, image)
        
        i = 0
        result_segmentation = []

        for i in range(6):
            result_segmentation1 = []
            tmp_bbox = []
            if len(result[0][i]):
                for j in range(len(result[0][i])):
                    if len(result_segmentation1) == 0:
                        contours = measure.find_contours(np.array(result[1][i][j]), 0.5)
                        result_segmentation1.append([contours[0].tolist(),result[0][i][j][0:5].tolist()])
                        tmp_bbox.append(result[0][i][j][0:4].tolist())

                    else:
                        if check_bbox(result[0][i][j][0:4].tolist(),tmp_bbox):
                            contours = measure.find_contours(np.array(result[1][i][j]), 0.5)
                            result_segmentation1.append([contours[0].tolist(),result[0][i][j][0:5].tolist()])
                            tmp_bbox.append(result[0][i][j][0:4].tolist())                       


                category = category_map[i+1]
                bbox_dict = {
                    category:result_segmentation1
                }

                if len(result_segmentation1):
                    result_segmentation.append(bbox_dict)
        
        bbox_dict1[img] = result_segmentation


with open('結果を出力するパス', 'w') as f:
    json.dump(bbox_dict1, f, ensure_ascii=False)

